In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import torchvision
import os
import shutil
import matplotlib.pyplot as plt
from   torchvision import transforms
%matplotlib inline

##### 读取数据集

In [2]:
base_dir = r'./data/FourWeather/'
train_dir= os.path.join(base_dir,"train")
test_dir = os.path.join(base_dir,"test")

In [3]:
train_transform = transforms.Compose([                                # 将图片在输入管道上做出一定变化，完成数据增强
    transforms.Resize(224),    
    transforms.RandomCrop(192),           
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(0.2),
    transforms.ColorJitter(brightness=0.5),
    transforms.ColorJitter(contrast=0.5),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5,0.5,0.5],std=[0.5,0.5,0.5])
])

test_transform  = transforms.Compose([
    transforms.Resize((192,192)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5,0.5,0.5],std=[0.5,0.5,0.5])
])

In [4]:
BATCH_SIZE = 32

In [5]:
train_ds = torchvision.datasets.ImageFolder(
    os.path.join(base_dir,"train"),
    transform = train_transform
)

test_ds  = torchvision.datasets.ImageFolder(
    os.path.join(base_dir,"test"),
    transform = test_transform
)

train_dl = torch.utils.data.DataLoader(train_ds,batch_size=BATCH_SIZE,shuffle=True)
test_dl  = torch.utils.data.DataLoader(test_ds,batch_size=BATCH_SIZE,shuffle=True)

In [6]:
class_to_id = train_ds.class_to_idx
id_to_class = dict((v,k) for k,v in train_ds.class_to_idx.items())

##### 定义模型

In [7]:
model = torchvision.models.vgg16(pretrained=True) 
model = model.to("cuda")

In [8]:
for p in model.features.parameters():
    p.requirs_grad = False                        # 设置卷积层里面的参数不再参与梯度下降与更新

In [9]:
optim     = torch.optim.Adam(model.parameters(),lr=0.0001)
loss_fn   = nn.CrossEntropyLoss()
epoches   = 20
model.classifier[-1].out_features = 4             # 将输出分类数修改为目标数量 (本例为4)

In [10]:
# Decay LR by a factor of 0.1 every 7 epochs
from torch.optim import lr_scheduler
optimizer = torch.optim.Adam(model.classifier.parameters(), lr=0.001)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1) 
                            #  .MultiStepLR(optimizer,milestone,gamma) #optimizer是优化器 milestone是列表表示在哪一步以系数gamma进行lr衰减 
                            #                  milestone可以是[10,20,..]类型的列表
                            #  .ExponentialLR(optimizer,gamma=0.1)     #每一步都以系数gamma进行lr衰减一次

In [11]:
train_loss = []
train_acc  = []
test_loss  = []
test_acc   = []

##### 定义fit函数

In [12]:
def fit(model, trainloader, testloader, epoch):
    correct = 0
    total = 0
    running_loss = 0
    for x, y in trainloader:
        x,y    = x.to("cuda"),y.to("cuda")
        y_pred = model(x)
        loss = loss_fn(y_pred, y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        with torch.no_grad():
            y_pred = torch.argmax(y_pred, dim=1)
            correct += (y_pred == y).sum().item()
            total += y.size(0)
            running_loss += loss.item()
    exp_lr_scheduler.step()   #经过一个epoch scheduler就会运行一步 每运行七步 学习速率会修改一次  即为每隔7步 学习速率*=0.1
    epoch_loss = running_loss / len(trainloader.dataset)
    epoch_acc = correct / total
        
        
    test_correct = 0
    test_total = 0
    test_running_loss = 0 
    
    with torch.no_grad():
        for x, y in testloader:
            x,y    = x.to("cuda"),y.to("cuda")
            y_pred = model(x)
            loss = loss_fn(y_pred, y)
            y_pred = torch.argmax(y_pred, dim=1)
            test_correct += (y_pred == y).sum().item()
            test_total += y.size(0)
            test_running_loss += loss.item()
    
    epoch_test_loss = test_running_loss / len(testloader.dataset)
    epoch_test_acc = test_correct / test_total
    
        
    print('epoch: ', epoch, 
          'loss： ', round(epoch_loss, 3),
          'accuracy:', round(epoch_acc, 3),
          'test_loss： ', round(epoch_test_loss, 3),
          'test_accuracy:', round(epoch_test_acc, 3)
             )
        
    return epoch_loss, epoch_acc, epoch_test_loss, epoch_test_acc

##### 训练模型

In [13]:
for epoch in range(epoches):
    epoch_loss,epoch_acc,epoch_test_loss,epoch_test_acc = fit(model,train_dl,test_dl,epoch)
    train_loss.append(epoch_loss)
    train_acc.append(epoch_acc)
    test_loss.append(epoch_test_loss)
    test_acc.append(epoch_test_acc)

RuntimeError: CUDA out of memory. Tried to allocate 392.00 MiB (GPU 0; 6.00 GiB total capacity; 818.25 MiB already allocated; 127.06 MiB free; 834.00 MiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
plt.plot(range(1,epoches+1),train_loss,label = "train_loss")
plt.plot(range(1,epoches+1),test_loss,label = "test_loss")
plt.legend()

In [ ]:
plt.plot(range(1, epoches+1), train_acc, label='train_acc')
plt.plot(range(1, epoches+1), test_acc, label='test_acc')
plt.legend()